## UHi Prediction Geographical data extraction notebook

- extracts features like building height max, mean, agg features, ring features, ground ele features from kml and shp files

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/test-df-ey-open-science/Submission_template_UHI2025-v2.csv
/kaggle/input/ey-geographical-shape-kml/Building_Footprint.kml
/kaggle/input/ey-geographical-shape-kml/BUILDING_view_-5453825549351750874/BUILDING.shp
/kaggle/input/ey-geographical-shape-kml/BUILDING_view_-5453825549351750874/BUILDING.shx
/kaggle/input/ey-geographical-shape-kml/BUILDING_view_-5453825549351750874/BUILDING.dbf
/kaggle/input/ey-geographical-shape-kml/BUILDING_view_-5453825549351750874/BUILDING.shp.xml
/kaggle/input/ey-geographical-shape-kml/BUILDING_view_-5453825549351750874/BUILDING.cpg
/kaggle/input/ey-geographical-shape-kml/BUILDING_view_-5453825549351750874/BUILDING.prj
/kaggle/input/uhi-index-data/Training_data_uhi_index_UHI2025-v2.csv


In [2]:
!pip install dask dask-geopandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 27.1 MB/s eta 0:00:0000:01
  Attempting uninstall: dask
    Found existing installation: dask 2024.11.2
    Uninstalling dask-2024.11.2:
      Successfully uninstalled dask-2024.11.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-expr 1.1.19 requires dask==2024.11.2, but you have dask 2025.2.0 which is incompatible.
distributed 2024.11.2 requires dask==2024.11.2, but you have dask 2025.2.0 which is incompatible.
rapids-dask-dependency 24.12.0 requires dask==2024.11.2, but you have dask 2025.2.0 which is incompatible.


## 2. shp file

In [ ]:
buildings_shp = gpd.read_file("/kaggle/input/ey-geographical-shape-kml/BUILDING_view_-5453825549351750874/BUILDING.shp")
buildings_shp.head(3)

In [ ]:
print("Building footprints loaded:", len(buildings_shp), "features")
buildings_shp = buildings_shp.to_crs("EPSG:2263")

In [ ]:
buildings_shp = buildings_shp.rename(columns={"geometry": "bld_geom"})
buildings_shp = buildings_shp.set_geometry("bld_geom")
print("Loaded building footprints:", len(buildings_shp), "features")

In [ ]:
buildings_shp['area_m2'] = buildings_shp.geometry.area

In [5]:
train_df = pd.read_csv('/kaggle/input/uhi-index-data/Training_data_uhi_index_UHI2025-v2.csv')
test_df = pd.read_csv('/kaggle/input/test-df-ey-open-science/Submission_template_UHI2025-v2.csv')

In [ ]:
from tqdm import tqdm
import gc

def df_to_gdf(df, lon_col='Longitude', lat_col='Latitude'):
    """
    Converts a DataFrame with lon/lat columns to a GeoDataFrame in EPSG:4326,
    then reprojects it to EPSG:2263 for accurate meter-based operations.
    """
    geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
    gdf_4326 = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
    # Reproject to EPSG:2263 for accurate distance/area
    return gdf_4326.to_crs("EPSG:2263")

def reproject_buildings(buildings_gdf):
    """
    Ensures building footprints are in EPSG:2263, 
    and rename geometry column if needed to avoid conflicts.
    """
    # Reproject building footprints to EPSG:2263
    bld_2263 = buildings_gdf.to_crs("EPSG:2263").copy()
    # Rename geometry column to "bld_geom"
    if "geometry" in bld_2263.columns:
        bld_2263 = bld_2263.rename(columns={"geometry": "bld_geom"})
        bld_2263 = bld_2263.set_geometry("bld_geom")
    return bld_2263

# def compute_building_features(
#     gdf_points: gpd.GeoDataFrame,
#     gdf_buildings: gpd.GeoDataFrame,
#     buffer_m: float = 100,
#     reproject_back: bool = True
# ):
#     """
#     1) Expects gdf_points and gdf_buildings in EPSG:2263 (projected).
#     2) Buffers each point by 'buffer_m' meters.
#     3) Spatially joins building footprints in that buffer.
#     4) Computes:
#        - building_count_<buffer_m>: # footprints
#        - building_area_<buffer_m>: total building area (m²)
#        - building_height_mean_<buffer_m>: average building height
#        - building_height_max_<buffer_m>: max building height
#        - ground_ele_mean_<buffer_m>: average ground elevation
#     5) Optionally reprojects points back to EPSG:4326.
#     """
#     gdf_points = gdf_points.copy()
#     # 1) Buffer
#     gdf_points["buffer"] = gdf_points.geometry.buffer(buffer_m)

#     # 2) Spatial join
#     joined = gpd.sjoin(
#         gdf_points.set_geometry("buffer"),
#         gdf_buildings,
#         how="left",
#         predicate="intersects",
#         lsuffix="pt",
#         rsuffix="bld"
#     )

#     # 3) Group by original points (left side)
#     # building counts
#     bld_count = joined.groupby(joined.index).size()

#     # building area (sum of bld_geom col)
#     # If you renamed geometry to "bld_geom", after the join it might appear as "bld_geom"
#     # or "bld_geom_bld" or "bld_geom_right". Let's handle that:
#     possible_geom_cols = ["bld_geom_bld", "bld_geom_right", "bld_geom", "geometry"]
#     building_geom_col = None
#     for candidate in possible_geom_cols:
#         if candidate in joined.columns:
#             building_geom_col = candidate
#             break

#     bld_area = joined.groupby(joined.index)["area_m2"].sum()
#     # building height mean / max (if "HEIGHT_ROO" in joined)
#     if "HEIGHT_ROO" in joined.columns:
#         bld_height_mean = joined.groupby(joined.index)["HEIGHT_ROO"].mean()
#         bld_height_max = joined.groupby(joined.index)["HEIGHT_ROO"].max()
#     else:
#         bld_height_mean = pd.Series(0, index=gdf_points.index)
#         bld_height_max  = pd.Series(0, index=gdf_points.index)

#     # ground elevation mean
#     if "GROUND_ELE" in joined.columns:
#         ground_ele_mean = joined.groupby(joined.index)["GROUND_ELE"].mean()
#     else:
#         ground_ele_mean = pd.Series(0, index=gdf_points.index)

#     # 4) Merge back to original points DataFrame
#     gdf_points[f"building_area_{buffer_m}m"] = bld_area.reindex(gdf_points.index, fill_value=0).values
#     gdf_points[f"building_count_{int(buffer_m)}m"] = bld_count.reindex(gdf_points.index, fill_value=0).values
#     gdf_points[f"building_area_{int(buffer_m)}m"] = bld_area.reindex(gdf_points.index, fill_value=0).values
#     gdf_points[f"building_height_mean_{int(buffer_m)}m"] = bld_height_mean.reindex(gdf_points.index, fill_value=0).values
#     gdf_points[f"building_height_max_{int(buffer_m)}m"]  = bld_height_max.reindex(gdf_points.index, fill_value=0).values
#     gdf_points[f"ground_ele_mean_{int(buffer_m)}m"]      = ground_ele_mean.reindex(gdf_points.index, fill_value=0).values

#     # Drop buffer
#     gdf_points.drop(columns=["buffer"], inplace=True)

#     # 5) Optionally reproject back to EPSG:4326
#     if reproject_back:
#         gdf_points = gdf_points.to_crs("EPSG:4326")

#     return gdf_points

# def compute_building_features_multi(gdf_points: gpd.GeoDataFrame,
#                                     gdf_buildings: gpd.GeoDataFrame,
#                                     buffer_list: list,
#                                     reproject_back: bool = True):
#     """
#     For each buffer size in buffer_list, this function computes building features by:
#       1) Creating an outer buffer of radius 'buffer_m'.
#       2) Creating an inner buffer (half the distance) and then computing the ring (outer minus inner).
#       3) For the outer buffer, it computes:
#            - building_count_{buffer_m}m: number of building footprints (dummy sum)
#            - building_area_{buffer_m}m: total building area (sum of area_m2)
#            - building_height_mean_{buffer_m}m: mean building height (if available)
#            - building_height_max_{buffer_m}m: maximum building height (if available)
#            - ground_ele_mean_{buffer_m}m: mean ground elevation (if available)
#          And additionally:
#            - building_area_min_{buffer_m}m, building_area_median_{buffer_m}m, building_area_std_{buffer_m}m
#            - building_height_median_{buffer_m}m, building_height_std_{buffer_m}m (if available)
#            - ground_ele_min_{buffer_m}m, ground_ele_median_{buffer_m}m, ground_ele_std_{buffer_m}m (if available)
#       4) For the ring area (outer buffer minus inner buffer), similar aggregations are computed with a "_ring" suffix.
#       5) All new columns are merged back into the original gdf_points.
#       6) Optionally reprojects the final GeoDataFrame back to EPSG:4326.
    
#     This function preserves your original column naming for the basic aggregations while adding new ones.
#     """
#     gdf_result = gdf_points.copy()
#     new_cols = {}
    
#     # Check for optional columns in gdf_buildings
#     has_height = "HEIGHT_ROO" in gdf_buildings.columns
#     has_ground = "GROUND_ELE" in gdf_buildings.columns

#     for buffer_m in tqdm(buffer_list, desc="Processing buffer sizes"):
#         # ---- Outer Buffer Aggregation ----
#         outer_buffer = gdf_result.geometry.buffer(buffer_m)
#         temp_outer = gdf_result.copy()
#         temp_outer = temp_outer.assign(buffer=outer_buffer)
#         joined_outer = gpd.sjoin(temp_outer.set_geometry("buffer"),
#                                  gdf_buildings,
#                                  how="left",
#                                  predicate="intersects",
#                                  lsuffix="pt",
#                                  rsuffix="bld")
#         # Dummy column for count
#         joined_outer["dummy"] = 1
        
#         # Define aggregation dictionary for outer buffer:
#         agg_dict = {
#             "dummy": "sum",               # building count
#             "area_m2": ["sum", "min", "median", "std"]  # building area aggregations
#         }
#         if has_height:
#             agg_dict["HEIGHT_ROO"] = ["mean", "max", "median", "std"]
#         if has_ground:
#             agg_dict["GROUND_ELE"] = ["mean", "min", "median", "std"]
        
#         agg_outer = joined_outer.groupby(joined_outer.index).agg(agg_dict)
#         # Flatten MultiIndex columns
#         agg_outer.columns = agg_outer.columns.to_flat_index()
        
#         # For outer buffer, we want to preserve the original naming for basic features:
#         # building_count, building_area (sum), building_height_mean, building_height_max, ground_ele_mean
#         # And add new ones with suffixes: _min, _median, _std for area; _median, _std for height; _min, _median, _std for ground.
#         # Create a mapping for outer buffer:
#         rename_map_outer = {}
#         # For building count
#         rename_map_outer[("dummy", "sum")] = f"building_count_{int(buffer_m)}m"
#         # For area_m2
#         rename_map_outer[("area_m2", "sum")] = f"building_area_{int(buffer_m)}m"
#         rename_map_outer[("area_m2", "min")] = f"building_area_min_{int(buffer_m)}m"
#         rename_map_outer[("area_m2", "median")] = f"building_area_median_{int(buffer_m)}m"
#         rename_map_outer[("area_m2", "std")] = f"building_area_std_{int(buffer_m)}m"
#         # For building height (if available)
#         if has_height:
#             rename_map_outer[("HEIGHT_ROO", "mean")] = f"building_height_mean_{int(buffer_m)}m"
#             rename_map_outer[("HEIGHT_ROO", "max")] = f"building_height_max_{int(buffer_m)}m"
#             rename_map_outer[("HEIGHT_ROO", "median")] = f"building_height_median_{int(buffer_m)}m"
#             rename_map_outer[("HEIGHT_ROO", "std")] = f"building_height_std_{int(buffer_m)}m"
#         # For ground elevation (if available)
#         if has_ground:
#             rename_map_outer[("GROUND_ELE", "mean")] = f"ground_ele_mean_{int(buffer_m)}m"
#             rename_map_outer[("GROUND_ELE", "min")] = f"ground_ele_min_{int(buffer_m)}m"
#             rename_map_outer[("GROUND_ELE", "median")] = f"ground_ele_median_{int(buffer_m)}m"
#             rename_map_outer[("GROUND_ELE", "std")] = f"ground_ele_std_{int(buffer_m)}m"
        
#         agg_outer = agg_outer.rename(columns=rename_map_outer)
#         agg_outer = agg_outer.reindex(gdf_result.index, fill_value=0)
        
#         # ---- Ring Buffer Aggregation ----
#         # Define inner buffer as half the outer radius (you can adjust this ratio if desired)
#         inner_buffer = gdf_result.geometry.buffer(buffer_m / 2)
#         ring_geom = outer_buffer.difference(inner_buffer)
        
#         temp_ring = gdf_result.copy()
#         temp_ring = temp_ring.assign(ring=ring_geom)
#         joined_ring = gpd.sjoin(temp_ring.set_geometry("ring"),
#                                 gdf_buildings,
#                                 how="left",
#                                 predicate="intersects",
#                                 lsuffix="pt",
#                                 rsuffix="bld")
#         joined_ring["dummy"] = 1
        
#         agg_ring = joined_ring.groupby(joined_ring.index).agg(agg_dict)
#         agg_ring.columns = agg_ring.columns.to_flat_index()
#         # Create renaming mapping for ring features (append '_ring' to the end)
#         rename_map_ring = {}
#         rename_map_ring[("dummy", "sum")] = f"building_count_{int(buffer_m)}m_ring"
#         rename_map_ring[("area_m2", "sum")] = f"building_area_{int(buffer_m)}m_ring"
#         rename_map_ring[("area_m2", "min")] = f"building_area_min_{int(buffer_m)}m_ring"
#         rename_map_ring[("area_m2", "median")] = f"building_area_median_{int(buffer_m)}m_ring"
#         rename_map_ring[("area_m2", "std")] = f"building_area_std_{int(buffer_m)}m_ring"
#         if has_height:
#             rename_map_ring[("HEIGHT_ROO", "mean")] = f"building_height_mean_{int(buffer_m)}m_ring"
#             rename_map_ring[("HEIGHT_ROO", "max")] = f"building_height_max_{int(buffer_m)}m_ring"
#             rename_map_ring[("HEIGHT_ROO", "median")] = f"building_height_median_{int(buffer_m)}m_ring"
#             rename_map_ring[("HEIGHT_ROO", "std")] = f"building_height_std_{int(buffer_m)}m_ring"
#         if has_ground:
#             rename_map_ring[("GROUND_ELE", "mean")] = f"ground_ele_mean_{int(buffer_m)}m_ring"
#             rename_map_ring[("GROUND_ELE", "min")] = f"ground_ele_min_{int(buffer_m)}m_ring"
#             rename_map_ring[("GROUND_ELE", "median")] = f"ground_ele_median_{int(buffer_m)}m_ring"
#             rename_map_ring[("GROUND_ELE", "std")] = f"ground_ele_std_{int(buffer_m)}m_ring"
        
#         agg_ring = agg_ring.rename(columns=rename_map_ring)
#         agg_ring = agg_ring.reindex(gdf_result.index, fill_value=0)
        
#         # Merge outer and ring features into new_cols dictionary.
#         for col in agg_outer.columns:
#             new_cols[col] = agg_outer[col].values
#         for col in agg_ring.columns:
#             new_cols[col] = agg_ring[col].values
        
#         # Clean up temporary variables
#         del outer_buffer, inner_buffer, ring_geom, temp_outer, temp_ring, joined_outer, joined_ring, agg_outer, agg_ring
#         gc.collect()
    
#     # Add all new columns from new_cols into gdf_result
#     for col, values in new_cols.items():
#         gdf_result[col] = values
    
#     if reproject_back:
#         gdf_result = gdf_result.to_crs("EPSG:4326")
    
#     return gdf_result

def compute_building_features_multi(gdf_points: gpd.GeoDataFrame,
                                    gdf_buildings: gpd.GeoDataFrame,
                                    buffer_list: list,
                                    reproject_back: bool = True,
                                    chunk_size: int = 500):
    """
    Optimized version with bulk column operations to prevent fragmentation
    """
    if 'area_m2' not in gdf_buildings.columns:
        raise ValueError("Buildings GeoDataFrame must contain 'area_m2' column")

    chunks = [
        gdf_points.iloc[i:i + chunk_size].copy()
        for i in range(0, len(gdf_points), chunk_size)
    ]
    
    processed_chunks = []
    
    for chunk in tqdm(chunks, desc="Total progress"):
        chunk_result = chunk.copy()
        all_new_data = {}
        
        # Pre-calculate buffers
        buffer_data = {
            buf: {
                'outer': chunk_result.geometry.buffer(buf),
                'inner': chunk_result.geometry.buffer(buf / 2)
            }
            for buf in buffer_list
        }
        
        for buffer_m in tqdm(buffer_list, desc="Buffer sizes", leave=False):
            buf_data = buffer_data[buffer_m]
            outer_buffer = buf_data['outer']
            ring_buffer = outer_buffer.difference(buf_data['inner'])
            
            # Process buffers
            outer_joined = gpd.sjoin(
                chunk_result.set_geometry(outer_buffer),
                gdf_buildings,
                how='left',
                predicate='intersects',
                lsuffix='pt',
                rsuffix='bld'
            ).assign(dummy=1)
            
            ring_joined = gpd.sjoin(
                chunk_result.set_geometry(ring_buffer),
                gdf_buildings,
                how='left',
                predicate='intersects',
                lsuffix='pt',
                rsuffix='bld'
            ).assign(dummy=1)
            
            # Define aggregations
            agg_dict = {
                'dummy': 'sum',
                'area_m2': ['sum', 'min', 'median', 'std']
            }
            if 'HEIGHT_ROO' in gdf_buildings:
                agg_dict['HEIGHT_ROO'] = ['mean', 'max', 'median', 'std']
            if 'GROUND_ELE' in gdf_buildings:
                agg_dict['GROUND_ELE'] = ['mean', 'min', 'median', 'std']
            
            # Bulk process aggregations
            def process_agg(joined, suffix):
                agg = joined.groupby(level=0).agg(agg_dict)
                agg.columns = agg.columns.to_flat_index()
                
                col_map = {
                    ('dummy', 'sum'): f'building_count_{buffer_m}m{suffix}',
                    ('area_m2', 'sum'): f'building_area_{buffer_m}m{suffix}',
                    ('area_m2', 'min'): f'building_area_min_{buffer_m}m{suffix}',
                    ('area_m2', 'median'): f'building_area_median_{buffer_m}m{suffix}',
                    ('area_m2', 'std'): f'building_area_std_{buffer_m}m{suffix}'
                }
                
                if 'HEIGHT_ROO' in gdf_buildings:
                    col_map.update({
                        ('HEIGHT_ROO', 'mean'): f'building_height_mean_{buffer_m}m{suffix}',
                        ('HEIGHT_ROO', 'max'): f'building_height_max_{buffer_m}m{suffix}',
                        ('HEIGHT_ROO', 'median'): f'building_height_median_{buffer_m}m{suffix}',
                        ('HEIGHT_ROO', 'std'): f'building_height_std_{buffer_m}m{suffix}'
                    })
                    
                if 'GROUND_ELE' in gdf_buildings:
                    col_map.update({
                        ('GROUND_ELE', 'mean'): f'ground_ele_mean_{buffer_m}m{suffix}',
                        ('GROUND_ELE', 'min'): f'ground_ele_min_{buffer_m}m{suffix}',
                        ('GROUND_ELE', 'median'): f'ground_ele_median_{buffer_m}m{suffix}',
                        ('GROUND_ELE', 'std'): f'ground_ele_std_{buffer_m}m{suffix}'
                    })
                
                return agg.rename(columns=col_map).reindex(chunk_result.index, fill_value=0)
            
            # Get aggregated data
            outer_data = process_agg(outer_joined, '')
            ring_data = process_agg(ring_joined, '_ring')
            
            # Merge results
            all_new_data.update(outer_data.to_dict('series'))
            all_new_data.update(ring_data.to_dict('series'))
            
            del outer_joined, ring_joined, outer_data, ring_data
            gc.collect()
        
        # Add all columns at once
        if all_new_data:
            new_cols_df = pd.DataFrame(all_new_data, index=chunk_result.index)
            chunk_result = pd.concat([chunk_result, new_cols_df], axis=1)
        
        processed_chunks.append(chunk_result)
        del chunk_result, new_cols_df
        gc.collect()
    
    # Final merge
    final_gdf = gpd.GeoDataFrame(
        pd.concat(processed_chunks, ignore_index=False),
        crs=chunks[0].crs if chunks else None
    )
    
    return final_gdf.to_crs("EPSG:4326") if reproject_back else final_gdf
# train_gdf_2263 = df_to_gdf(train_df)
# test_gdf_2263 = df_to_gdf(test_df)

In [ ]:
# import geopandas as gpd
# import pandas as pd
# from shapely.geometry import Point

# def df_to_gdf(df, lon_col='Longitude', lat_col='Latitude'):
#     """
#     Converts a DataFrame with lon/lat columns to a GeoDataFrame in EPSG:4326,
#     then reprojects it to EPSG:2263 for accurate meter-based operations.
#     """
#     geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
#     gdf_4326 = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
#     # Reproject to EPSG:2263 for accurate distance/area
#     return gdf_4326.to_crs("EPSG:2263")

# def reproject_buildings(buildings_gdf):
#     """
#     Ensures building footprints are in EPSG:2263, 
#     and rename geometry column if needed to avoid conflicts.
#     """
#     # Reproject building footprints to EPSG:2263
#     bld_2263 = buildings_gdf.to_crs("EPSG:2263").copy()
#     # Rename geometry column to "bld_geom"
#     if "geometry" in bld_2263.columns:
#         bld_2263 = bld_2263.rename(columns={"geometry": "bld_geom"})
#         bld_2263 = bld_2263.set_geometry("bld_geom")
#     return bld_2263

# def compute_building_features(
#     gdf_points: gpd.GeoDataFrame,
#     gdf_buildings: gpd.GeoDataFrame,
#     buffer_m: float = 100,
#     reproject_back: bool = True
# ):
#     """
#     1) Expects gdf_points and gdf_buildings in EPSG:2263 (projected).
#     2) Buffers each point by 'buffer_m' meters.
#     3) Spatially joins building footprints in that buffer.
#     4) Computes:
#        - building_count_<buffer_m>: # footprints
#        - building_area_<buffer_m>: total building area (m²)
#        - building_height_mean_<buffer_m>: average building height
#        - building_height_max_<buffer_m>: max building height
#        - ground_ele_mean_<buffer_m>: average ground elevation
#        - building_age_mean_<buffer_m>: mean building age
#        - building_age_max_<buffer_m>: max building age
#        - geom_source_mode_<buffer_m>: most common geometry source
#        - feature_co_mode_<buffer_m>: most common feature code
#     5) Optionally reprojects points back to EPSG:4326.
#     """
#     gdf_points = gdf_points.copy()
    
#     # 1) Buffer
#     gdf_points["buffer"] = gdf_points.geometry.buffer(buffer_m)

#     # 2) Spatial join
#     building_geom_col = gdf_buildings.geometry.name  # Get original name (e.g., "bld_geom")
#     joined = gpd.sjoin(
#         gdf_points.set_geometry("buffer"),
#         gdf_buildings,
#         how="left",
#         predicate="intersects",
#         lsuffix="pt",
#         rsuffix="bld"
#     )

#     # 3) Group by original points (left side)
#     grouped = joined.groupby(joined.index)

#     # Core metrics
#     gdf_points[f"building_count_{buffer_m}m"] = grouped.size().reindex(gdf_points.index, fill_value=0)
    
#     # Area (fixed)
#     building_geom_col_joined = f"{building_geom_col}_bld"  # e.g., "bld_geom_bld"
#     if building_geom_col_joined in joined.columns:
#         gdf_points[f"building_area_{buffer_m}m"] = grouped[building_geom_col_joined].apply(
#             lambda geoms: sum(
#                 g.area 
#                 for g in geoms 
#                 if g is not None and not g.is_empty and g.geom_type == "Polygon"
#             )
#         ).reindex(gdf_points.index, fill_value=0)
#     else:
#         gdf_points[f"building_area_{buffer_m}m"] = 0

#     # Extended features
#     if "HEIGHT_ROO" in joined.columns:
#         gdf_points[f"bld_height_max_{buffer_m}m"] = grouped["HEIGHT_ROO"].max().fillna(0)
#         gdf_points[f"bld_height_mean_{buffer_m}m"] = grouped["HEIGHT_ROO"].mean().fillna(0)
    
#     if "GROUND_ELE" in joined.columns:
#         gdf_points[f"ground_ele_mean_{buffer_m}m"] = grouped["GROUND_ELE"].mean().fillna(0)
    
#     if "CONSTRUCTI" in joined.columns:
#         gdf_points[f"bld_age_mean_{buffer_m}m"] = (2023 - grouped["CONSTRUCTI"].mean()).fillna(0)
#         gdf_points[f"bld_age_max_{buffer_m}m"] = (2023 - grouped["CONSTRUCTI"].min()).fillna(0)
    
#     # Categorical features (mode)
#     for cat_col in ["GEOM_SOURC", "FEATURE_CO", "LAST_STATU"]:
#         if cat_col in joined.columns:
#             gdf_points[f"{cat_col.lower()}_mode_{buffer_m}m"] = grouped[cat_col].agg(
#                 lambda x: x.mode()[0] if not x.empty else "None"
#             )

#     # Drop buffer
#     gdf_points.drop(columns=["buffer"], inplace=True)

#     # 5) Optionally reproject back to EPSG:4326
#     if reproject_back:
#         gdf_points = gdf_points.to_crs("EPSG:4326")

#     return gdf_points

In [ ]:
train_gdf_2263 = df_to_gdf(train_df)
test_gdf_2263 = df_to_gdf(test_df)

In [ ]:
bld_2263 = reproject_buildings(buildings_shp)

In [ ]:
# Count geometry types in the building data
geometry_counts = buildings_shp.geometry.type.value_counts()
print("Geometry Types in Building Data:\n", geometry_counts)


In [ ]:
# import matplotlib.pyplot as plt
# import geopandas as gpd

# # Select a sample point
# sample_point = train_gdf_2263.iloc[0]  
# buffer = sample_point.geometry.buffer(1500)  # 100m buffer

# # Convert to GeoSeries for plotting
# buffer_gs = gpd.GeoSeries(buffer)

# # Plotting the buffer, buildings, and sample point
# fig, ax = plt.subplots(figsize=(10, 10))

# # Plot the buffer boundary
# buffer_gs.boundary.plot(ax=ax, color='red', linewidth=2, label='Buffer Boundary')

# # Plot the buildings
# gdf_buildings_clean.plot(ax=ax, facecolor='none', edgecolor='blue', alpha=0.5, label='Buildings')

# # Plot the sample point
# plt.scatter(sample_point.geometry.x, sample_point.geometry.y, color='green', s=100, label='Sample Point')

# plt.legend()
# plt.show()


In [ ]:
buffer_shp_list = [10000, 9500, 9000, 8500, 8000,
                    7500, 7000, 6500, 6000, 5500, 5000, 4500, 4000, 3500,
                    3000, 2500, 2000, 1500, 1000, 500, 400, 300, 250, 150, 100
                  ]

In [ ]:
train_gdf_features = compute_building_features_multi(train_gdf_2263, bld_2263, buffer_list=buffer_shp_list, reproject_back=True)
test_gdf_features = compute_building_features_multi(test_gdf_2263, bld_2263, buffer_list=buffer_shp_list, reproject_back=True)

In [ ]:
train_gdf_features.columns

In [ ]:
# train_gdf_features.columns

In [ ]:
# train_gdf_1500 = compute_building_features(train_gdf_2263, bld_2263, buffer_m=1500, reproject_back=True)
# test_gdf_1500  = compute_building_features(test_gdf_2263,  bld_2263, buffer_m=1500, reproject_back=True)
# train_gdf_1000 = compute_building_features(train_gdf_2263, bld_2263, buffer_m=1000, reproject_back=True)
# test_gdf_1000  = compute_building_features(test_gdf_2263,  bld_2263, buffer_m=1000, reproject_back=True)
# train_gdf_500 = compute_building_features(train_gdf_2263, bld_2263, buffer_m=500, reproject_back=True)
# test_gdf_500  = compute_building_features(test_gdf_2263,  bld_2263, buffer_m=500, reproject_back=True)
# train_gdf_250 = compute_building_features(train_gdf_2263, bld_2263, buffer_m=250, reproject_back=True)
# test_gdf_250  = compute_building_features(test_gdf_2263,  bld_2263, buffer_m=250, reproject_back=True)
# train_gdf_100 = compute_building_features(train_gdf_2263, bld_2263, buffer_m=100, reproject_back=True)
# test_gdf_100  = compute_building_features(test_gdf_2263,  bld_2263, buffer_m=100, reproject_back=True)
# train_gdf_50 = compute_building_features(train_gdf_2263, bld_2263, buffer_m=50, reproject_back=True)
# test_gdf_50  = compute_building_features(test_gdf_2263,  bld_2263, buffer_m=50, reproject_back=True)

In [ ]:
# train_gdf_features.shape

In [ ]:
# cols_to_drop = ["Longitude", "Latitude", "datetime", "UHI Index", "geometry"]
# for df in [train_gdf_1500, train_gdf_1000, train_gdf_500, train_gdf_250, train_gdf_100, train_gdf_50]:
#     df.drop(columns=cols_to_drop, inplace=True)

In [ ]:
# cols_to_drop_test = [	'Longitude',	'Latitude',	'UHI Index',	'geometry']
# for df in [test_gdf_1500, test_gdf_1000, test_gdf_500, test_gdf_250, test_gdf_100, test_gdf_50]:
#     df.drop(columns=cols_to_drop_test, inplace=True)

In [ ]:
# train_gdf_features = pd.concat([train_gdf_50, train_gdf_100, train_gdf_250, train_gdf_500, train_gdf_1000, train_gdf_1500], axis=1)
# test_gdf_features = pd.concat([test_gdf_50, test_gdf_100, test_gdf_250, test_gdf_500, test_gdf_1000, test_gdf_1500], axis=1)

In [ ]:
# train_gdf_features
# cols_to_drop = ["Longitude", "Latitude", "datetime", "UHI Index", "geometry"]
# train_gdf_features.drop(columns=cols_to_drop, inplace=True)

In [ ]:
# cols_to_drop_test = [col for col in cols_to_drop if col!='UHI Index' and col!='building_coverage_100m' and col!='datetime']
# test_gdf_features.drop(columns=cols_to_drop_test, inplace=True)

In [ ]:
# test_gdf_features.drop(columns='UHI Index', inplace=True)

In [ ]:
# train_gdf_features["geom_sourc_mode_1500m"] = train_gdf_features["geom_sourc_mode_1500m"].astype("category")

In [ ]:
# for col in train_gdf_features.columns:
#     col_mean = np.mean(train_gdf_features[col])
#     train_gdf_features[col].fillna(col_mean, inplace=True)

In [ ]:
# train_gdf_features.isnull().sum()

In [ ]:
# for col in test_gdf_features.columns:
#     col_test_mean = np.mean(test_gdf_features[col])
#     test_gdf_features[col].fillna(col_mean, inplace=True)

In [ ]:
cols_to_drop = ['Longitude', 'Latitude', 'datetime', 'UHI Index', 'geometry']
cols_to_drop_test = ['Longitude', 'Latitude', 'UHI Index', 'geometry']
train_gdf_features.drop(columns=cols_to_drop, inplace=True)
test_gdf_features.drop(columns=cols_to_drop_test, inplace=True)

In [ ]:
# import xgboost as xgb
# input_cols = train_gdf_features.columns
# target = train_df["UHI Index"]
# xgb_reg = xgb.XGBRegressor(enable_categorical=False, random_state=42)
# xgb_reg.fit(train_gdf_features, target)

In [ ]:
# xgb_reg.get_booster().get_score(importance_type='weight')

In [ ]:
# preds = xgb_reg.predict(test_gdf_features)

# preds

In [ ]:
# subm_test = pd.read_csv('/kaggle/input/test-df-ey-open-science/Submission_template_UHI2025-v2.csv')
# subm_test['UHI Index'] = preds

In [ ]:
# subm_test.to_csv('building_features_shp(subm).csv', index=False)

In [ ]:
train_gdf_features.to_csv('train_shp.csv', index=False)
test_gdf_features.to_csv('test_shp.csv', index=False)

In [ ]:
# import pyogrio
# import geopandas as gpd

# # 1. Read the KML file using pyogrio
# buildings = pyogrio.read_dataframe("/kaggle/input/ey-geographical-shape-kml/Building_Footprint.kml")
# print("Initial CRS:", buildings.crs)
# print("Geometry types:\n", buildings.geometry.geom_type.value_counts())

# # 2. Reproject to EPSG:2263 for accurate meter-based area calculations
# buildings = buildings.to_crs("EPSG:2263")
# print("Reprojected CRS:", buildings.crs)

# # 3. Explode MultiPolygons into individual Polygons (if necessary)
# buildings = buildings.explode(index_parts=False).reset_index(drop=True)
# print("Geometry types after exploding:\n", buildings.geometry.geom_type.value_counts())

# # 4. Calculate area (in square meters) and add as a new column
# buildings["area_m2"] = buildings.geometry.area

# # 5. Display summary statistics of the area
# print(buildings[["area_m2"]].describe())

# # Optional: Save the processed building footprints with area to a new file
# buildings.to_file("building_footprints_processed.geojson", driver="GeoJSON")


In [ ]:
# # Convert DataFrame to GeoDataFrame and reproject:
# def df_to_gdf(df, lon_col='Longitude', lat_col='Latitude', crs='EPSG:4326'):
#     from shapely.geometry import Point
#     geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
#     gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=crs)
#     return gdf.to_crs("EPSG:2263")

# train_gdf = df_to_gdf(train_df)
# test_gdf  = df_to_gdf(test_df)

# # Create buffer (example: 1500m)
# buffer_m = 1500
# train_gdf["buffer"] = train_gdf.geometry.buffer(buffer_m)

# # Spatial join: join buffered points with building footprints
# joined = gpd.sjoin(
#     train_gdf.set_geometry("buffer"),
#     buildings,  # the processed building GeoDataFrame with 'area_m2'
#     how="left",
#     predicate="intersects",
#     rsuffix="_bld"
# )

# # Group by original point index and sum building areas
# bld_area = joined.groupby(joined.index)["area_m2"].sum()
# bld_count = joined.groupby(joined.index).size()
# # Merge back to train_gdf
# train_gdf[f"building_area_{buffer_m}m_kml"] = bld_area.reindex(train_gdf.index, fill_value=0).values
# train_gdf[f'building_count_{buffer_m}m_kml'] = bld_count.reindex(train_gdf.index, fill_value=0).values
# train_gdf.drop(columns=["buffer"], inplace=True)

# # Optionally reproject back to EPSG:4326:
# train_final = train_gdf.to_crs("EPSG:4326")

# print(train_final[[f"building_area_{buffer_m}m_kml"]].describe())


In [ ]:
# import geopandas as gpd
# import pandas as pd
# from shapely.geometry import Point

# # Function to convert a DataFrame to a GeoDataFrame in EPSG:2263
# def df_to_gdf(df, lon_col='Longitude', lat_col='Latitude', crs='EPSG:4326'):
#     geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
#     gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=crs)
#     return gdf.to_crs("EPSG:2263")

# # Convert train and test DataFrames
# train_gdf = df_to_gdf(train_df)
# test_gdf  = df_to_gdf(test_df)

# # Buffer size in meters
# buffer_m = 11000

# ### --- Process TRAIN Data ---
# train_gdf["buffer"] = train_gdf.geometry.buffer(buffer_m)
# joined_train = gpd.sjoin(
#     train_gdf.set_geometry("buffer"),
#     buildings,  # processed building GeoDataFrame with 'area_m2'
#     how="left",
#     predicate="intersects",
#     rsuffix="_bld"
# )
# bld_area_train = joined_train.groupby(joined_train.index)["area_m2"].sum()
# bld_count_train = joined_train.groupby(joined_train.index).size()
# train_gdf[f"building_area_{buffer_m}m_kml"] = bld_area_train.reindex(train_gdf.index, fill_value=0).values
# train_gdf[f"building_count_{buffer_m}m_kml"] = bld_count_train.reindex(train_gdf.index, fill_value=0).values
# train_gdf.drop(columns=["buffer"], inplace=True)
# train_final = train_gdf.to_crs("EPSG:4326")
# print("Train building area summary:")
# print(train_final[[f"building_area_{buffer_m}m_kml"]].describe())

# ### --- Process TEST Data ---
# test_gdf["buffer"] = test_gdf.geometry.buffer(buffer_m)
# joined_test = gpd.sjoin(
#     test_gdf.set_geometry("buffer"),
#     buildings,  # same processed building GeoDataFrame
#     how="left",
#     predicate="intersects",
#     rsuffix="_bld"
# )
# bld_area_test = joined_test.groupby(joined_test.index)["area_m2"].sum()
# bld_count_test = joined_test.groupby(joined_test.index).size()
# test_gdf[f"building_area_{buffer_m}m_kml"] = bld_area_test.reindex(test_gdf.index, fill_value=0).values
# test_gdf[f"building_count_{buffer_m}m_kml"] = bld_count_test.reindex(test_gdf.index, fill_value=0).values
# test_gdf.drop(columns=["buffer"], inplace=True)
# test_final = test_gdf.to_crs("EPSG:4326")
# print("Test building area summary:")
# print(test_final[[f"building_area_{buffer_m}m_kml"]].describe())


In [ ]:
# import geopandas as gpd
# from shapely.geometry import Point
# import pandas as pd
# import pyogrio
# # (Assume these functions are defined as in your code.)
# def df_to_gdf(df, lon_col='Longitude', lat_col='Latitude'):
#     """
#     Converts a DataFrame with lon/lat columns to a GeoDataFrame in EPSG:4326,
#     then reprojects it to EPSG:2263 for accurate meter-based operations.
#     """
#     geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
#     gdf_4326 = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
#     # Reproject to EPSG:2263 for accurate distance/area calculations
#     return gdf_4326.to_crs("EPSG:2263")

# def reproject_buildings(buildings_gdf):
#     """
#     Ensures building footprints are in EPSG:2263 and renames the geometry column to avoid conflicts.
#     """
#     bld_2263 = buildings_gdf.to_crs("EPSG:2263").copy()
#     if "geometry" in bld_2263.columns:
#         bld_2263 = bld_2263.rename(columns={"geometry": "bld_geom"})
#         bld_2263 = bld_2263.set_geometry("bld_geom")
#     return bld_2263

# def compute_building_features(
#     gdf_points: gpd.GeoDataFrame,
#     gdf_buildings: gpd.GeoDataFrame,
#     buffer_m: float = 100,
#     reproject_back: bool = True
# ):
#     """
#     For each point in gdf_points (in EPSG:2263), buffers by 'buffer_m' meters and 
#     spatially joins the building footprints from gdf_buildings.
#     Computes the following new columns:
#       - building_area_{buffer_m}m : total building area (m²) within the buffer
#       - building_count_{buffer_m}m : number of building footprints intersecting the buffer
#       - building_height_mean_{buffer_m}m : mean building height (if available, else 0)
#       - building_height_max_{buffer_m}m : maximum building height (if available, else 0)
#       - ground_ele_mean_{buffer_m}m     : mean ground elevation (if available, else 0)
#     Optionally reprojects back to EPSG:4326.
#     """
#     gdf_points = gdf_points.copy()
#     # 1) Buffer each point
#     gdf_points["buffer"] = gdf_points.geometry.buffer(buffer_m)

#     # 2) Spatial join with buildings
#     joined = gpd.sjoin(
#         gdf_points.set_geometry("buffer"),
#         gdf_buildings,
#         how="left",
#         predicate="intersects",
#         lsuffix="pt",
#         rsuffix="bld"
#     )

#     # 3) Group by the original point indices to compute counts and sums
#     bld_count = joined.groupby(joined.index).size()
#     # We assume that the building footprints already have an "area_m2" column
#     bld_area = joined.groupby(joined.index)["area_m2"].sum()
#     # For height and ground elevation, check if the columns exist
#     if "HEIGHT_ROO" in joined.columns:
#         bld_height_mean = joined.groupby(joined.index)["HEIGHT_ROO"].mean()
#         bld_height_max = joined.groupby(joined.index)["HEIGHT_ROO"].max()
#     else:
#         bld_height_mean = pd.Series(0, index=gdf_points.index)
#         bld_height_max  = pd.Series(0, index=gdf_points.index)

#     if "GROUND_ELE" in joined.columns:
#         ground_ele_mean = joined.groupby(joined.index)["GROUND_ELE"].mean()
#     else:
#         ground_ele_mean = pd.Series(0, index=gdf_points.index)

#     # 4) Create new columns (we use the buffer size in the column names)
#     buf_int = int(buffer_m)
#     gdf_points[f"building_area_{buf_int}m_kml"] = bld_area.reindex(gdf_points.index, fill_value=0).values
#     gdf_points[f"building_count_{buf_int}m_kml"] = bld_count.reindex(gdf_points.index, fill_value=0).values
#     # gdf_points[f"building_height_mean_{buf_int}m"] = bld_height_mean.reindex(gdf_points.index, fill_value=0).values
#     # gdf_points[f"building_height_max_{buf_int}m"]  = bld_height_max.reindex(gdf_points.index, fill_value=0).values
#     # gdf_points[f"ground_ele_mean_{buf_int}m"]      = ground_ele_mean.reindex(gdf_points.index, fill_value=0).values

#     # Drop the temporary buffer column and reset the geometry back to the original point geometry
#     gdf_points.drop(columns=["buffer"], inplace=True)
#     gdf_points.set_geometry("geometry", inplace=True)

#     # Optionally reproject back to EPSG:4326
#     if reproject_back:
#         gdf_points = gdf_points.to_crs("EPSG:4326")

#     return gdf_points

# def join_building_features(gdf_points: gpd.GeoDataFrame, gdf_buildings: gpd.GeoDataFrame, 
#                              buffer_sizes: list, reproject_back: bool = True):
#     """
#     Computes building features at multiple buffer sizes and merges the new columns into a single GeoDataFrame.
#     It does not duplicate any columns that already exist.
    
#     Args:
#         gdf_points: Base GeoDataFrame (in EPSG:2263) containing points (e.g., train or test data).
#         gdf_buildings: GeoDataFrame of building footprints with a precomputed "area_m2" column.
#         buffer_sizes: List of buffer distances (in meters) at which to compute features.
#         reproject_back: If True, reproject the final result back to EPSG:4326.
    
#     Returns:
#         GeoDataFrame with the original columns and the additional building features for each buffer.
#     """
#     # Start with a copy of the base points
#     base = gdf_points.copy()
#     for buf in buffer_sizes:
#         computed = compute_building_features(gdf_points, gdf_buildings, buffer_m=buf, reproject_back=False)
#         # Select only the new columns (those that contain the buffer value in their name)
#         new_cols = [col for col in computed.columns if f"_{int(buf)}m" in col]
#         # Join these new columns to the base dataframe
#         base = base.join(computed[new_cols])
#     # Optionally reproject back to EPSG:4326
#     if reproject_back:
#         base = base.to_crs("EPSG:4326")
#     return base

# # Example usage:
# # (Assume train_df and test_df are your original DataFrames with Latitude/Longitude columns)
# # Convert to GeoDataFrames in EPSG:2263
# train_gdf_2263 = df_to_gdf(train_df, lon_col='Longitude', lat_col='Latitude')
# test_gdf_2263  = df_to_gdf(test_df, lon_col='Longitude', lat_col='Latitude')

# # Load and reproject building footprints from your KML file
# buildings = pyogrio.read_dataframe("/kaggle/input/ey-geographical-shape-kml/Building_Footprint.kml")
# buildings_2263 = reproject_buildings(buildings)
# buildings_2263['area_m2'] = buildings_2263.geometry.area
# # Specify the list of buffer sizes (in meters) for which you want to compute features
# buffer_sizes = [ #15000, 14500, 14000, 13500, 13000, 12500, 12000, 11500, 
#                 # 11000,10500, 10000, 9500, 9000, 8500, 8000, 7500, 7000, 6500,
#                 # 6000, 5500, 5000, 4500, 4000, 3500,
#                 # 3000, 2500, 2000, 1500, 
#                 #1000, 500, 250, 100, 50]

# # Compute and merge building features for train and test data
# train_gdf_features = join_building_features(train_gdf_2263, buildings_2263, buffer_sizes, reproject_back=True)
# test_gdf_features  = join_building_features(test_gdf_2263, buildings_2263, buffer_sizes, reproject_back=True)

# # Convert GeoDataFrames to DataFrames (if needed)
# train_features_df = pd.DataFrame(train_gdf_features.drop(columns='geometry'))
# test_features_df  = pd.DataFrame(test_gdf_features.drop(columns='geometry'))

# # print("Train features:")
# # print(train_features_df.head())
# # print("Test features:")
# # print(test_features_df.head())


## 3. kml file

In [6]:
from tqdm import tqdm
import geopandas as gpd
import pandas as pd
import numpy as np
import pyogrio
import gc
from shapely.geometry import Point

def df_to_gdf(df, lon_col='Longitude', lat_col='Latitude'):
    """
    Converts a DataFrame with lon/lat columns to a GeoDataFrame in EPSG:4326,
    then reprojects it to EPSG:2263 for accurate meter-based operations.
    """
    geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
    gdf_4326 = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
    # Reproject to EPSG:2263 for accurate distance/area
    return gdf_4326.to_crs("EPSG:2263")

def reproject_buildings(buildings_gdf):
    """
    Ensures building footprints are in EPSG:2263, 
    and renames geometry column to avoid conflicts.
    """
    bld_2263 = buildings_gdf.to_crs("EPSG:2263").copy()
    if "geometry" in bld_2263.columns:
        bld_2263 = bld_2263.rename(columns={"geometry": "bld_geom"})
        bld_2263 = bld_2263.set_geometry("bld_geom")
    return bld_2263

def compute_building_features(gdf_points: gpd.GeoDataFrame,
                              gdf_buildings: gpd.GeoDataFrame,
                              buffer_m: float = 100,
                              reproject_back: bool = True):
    """
    For each point, buffers by 'buffer_m' meters and spatially joins the building footprints.
    Computes these basic features:
      - building_count_{buffer_m}m_kml: # footprints intersecting the buffer
      - building_area_{buffer_m}m_kml : total building area (sum of area_m2)
    This maintains your original naming conventions and 'kml' suffix.
    
    Optionally reprojects points back to EPSG:4326.
    """
    gdf_points = gdf_points.copy()
    # Create an outer buffer
    gdf_points["buffer"] = gdf_points.geometry.buffer(buffer_m)
    
    joined = gpd.sjoin(
        gdf_points.set_geometry("buffer"),
        gdf_buildings,  # must have 'area_m2' column
        how="left",
        predicate="intersects",
        lsuffix="pt",
        rsuffix="bld"
    )
    
    # Basic aggregations
    bld_count = joined.groupby(joined.index).size()
    bld_area_sum = joined.groupby(joined.index)["area_m2"].sum()
    
    # Add columns to gdf_points
    gdf_points[f"building_area_{int(buffer_m)}m_kml"]  = bld_area_sum.reindex(gdf_points.index, fill_value=0).values
    gdf_points[f"building_count_{int(buffer_m)}m_kml"] = bld_count.reindex(gdf_points.index, fill_value=0).values
    
    # Clean up
    gdf_points.drop(columns=["buffer"], inplace=True)
    gdf_points.set_geometry("geometry", inplace=True)
    
    if reproject_back:
        gdf_points = gdf_points.to_crs("EPSG:4326")
    return gdf_points

def compute_building_features_multi(gdf_points: gpd.GeoDataFrame,
                                    gdf_buildings: gpd.GeoDataFrame,
                                    buffer_list: list,
                                    reproject_back: bool = True):
    """
    For each buffer size in buffer_list, this function computes BOTH:
      (A) Outer buffer features (basic + advanced aggregations)
      (B) Ring (annulus) features by subtracting an inner buffer (half distance) from the outer buffer.
    
    Aggregations (for building area_m2):
      - sum (as your original code had)
      - min, median, std (new)
    building_count is also included (dummy sum).
    
    Naming:
      - Basic: building_count_{buffer_m}m_kml, building_area_{buffer_m}m_kml
      - Additional: building_area_min_{buffer_m}m_kml, building_area_median_{buffer_m}m_kml, building_area_std_{buffer_m}m_kml
      - Ring versions have a "_ring" suffix, e.g. building_count_{buffer_m}m_kml_ring, building_area_{buffer_m}m_kml_ring
    
    The new columns are merged into the original GeoDataFrame.
    """
    gdf_result = gdf_points.copy()
    new_cols = {}  # store newly computed columns so we can add them at once
    
    for buf in tqdm(buffer_list, desc="Processing KML building buffers"):
        print(f'current buffer being processed :{buf}')
        # === Outer Buffer ===
        outer_buffer = gdf_result.geometry.buffer(buf)
        temp_outer = gdf_result.copy()
        temp_outer["outer_buffer"] = outer_buffer
        
        joined_outer = gpd.sjoin(
            temp_outer.set_geometry("outer_buffer"),
            gdf_buildings,  # must have 'area_m2'
            how="left",
            predicate="intersects",
            lsuffix="pt",
            rsuffix="bld"
        )
        joined_outer["dummy"] = 1  # for counting footprints
        
        # Aggregations for outer buffer
        agg_dict = {
            "dummy": "sum",
            "area_m2": ["sum", "min", "median", "std"]
        }
        agg_outer = joined_outer.groupby(joined_outer.index).agg(agg_dict)
        agg_outer.columns = agg_outer.columns.to_flat_index()  # flatten multiindex
        
        # rename them to preserve your original naming + new suffixes
        rename_map_outer = {
            ("dummy", "sum"): f"building_count_{int(buf)}m_kml",
            ("area_m2", "sum"): f"building_area_{int(buf)}m_kml",
            ("area_m2", "min"): f"building_area_min_{int(buf)}m_kml",
            ("area_m2", "median"): f"building_area_median_{int(buf)}m_kml",
            ("area_m2", "std"): f"building_area_std_{int(buf)}m_kml",
        }
        agg_outer = agg_outer.rename(columns=rename_map_outer)
        agg_outer = agg_outer.reindex(gdf_result.index, fill_value=0)
        
        # === Ring Buffer (Annulus) ===
        inner_buffer = gdf_result.geometry.buffer(buf / 2)
        ring_geom = outer_buffer.difference(inner_buffer)
        
        temp_ring = gdf_result.copy()
        temp_ring["ring_buffer"] = ring_geom
        
        joined_ring = gpd.sjoin(
            temp_ring.set_geometry("ring_buffer"),
            gdf_buildings,
            how="left",
            predicate="intersects",
            lsuffix="pt",
            rsuffix="bld"
        )
        joined_ring["dummy"] = 1
        
        agg_ring = joined_ring.groupby(joined_ring.index).agg(agg_dict)
        agg_ring.columns = agg_ring.columns.to_flat_index()
        
        # rename ring columns
        rename_map_ring = {
            ("dummy", "sum"): f"building_count_{int(buf)}m_kml_ring",
            ("area_m2", "sum"): f"building_area_{int(buf)}m_kml_ring",
            ("area_m2", "min"): f"building_area_min_{int(buf)}m_kml_ring",
            ("area_m2", "median"): f"building_area_median_{int(buf)}m_kml_ring",
            ("area_m2", "std"): f"building_area_std_{int(buf)}m_kml_ring",
        }
        agg_ring = agg_ring.rename(columns=rename_map_ring)
        agg_ring = agg_ring.reindex(gdf_result.index, fill_value=0)
        
        # store columns from both outer and ring
        for col in agg_outer.columns:
            new_cols[col] = agg_outer[col].values
        for col in agg_ring.columns:
            new_cols[col] = agg_ring[col].values
        
        del temp_outer, joined_outer, temp_ring, joined_ring, agg_outer, agg_ring
        gc.collect()
    
    # add all new columns
    for col, vals in new_cols.items():
        gdf_result[col] = vals
    
    if reproject_back:
        gdf_result = gdf_result.to_crs("EPSG:4326")
    
    return gdf_result

def join_building_features(gdf_points, gdf_buildings, buffer_sizes, reproject_back=True):
    # Just one call to compute_building_features_multi to process ALL buffers
    computed = compute_building_features_multi(
        gdf_points, gdf_buildings, buffer_list=buffer_sizes, reproject_back=False
    )
    # Merge all newly created columns into the base DataFrame
    base = gdf_points.copy()
    new_cols = [c for c in computed.columns if c not in base.columns]
    base = base.join(computed[new_cols])
    if reproject_back:
        base = base.to_crs("EPSG:4326")
    return base

# Example usage:
# Suppose train_df and test_df are your original data with lat/long.
# 1) Convert them to EPSG:2263
train_gdf_2263 = df_to_gdf(train_df)
test_gdf_2263  = df_to_gdf(test_df)

# 2) Read building footprints from KML, reproject, compute 'area_m2'
buildings = pyogrio.read_dataframe("/kaggle/input/ey-geographical-shape-kml/Building_Footprint.kml")
buildings_2263 = reproject_buildings(buildings)
buildings_2263['area_m2'] = buildings_2263['bld_geom'].area

# 3) Decide on buffer sizes
buffer_sizes = [15000,
                 14500, 14000, 13500, 13000, 12500, 12000, 11500, 11000, 10500, 10000, 9500, 9000, 8500, 8000, 7500, 7000, 6500, 6000, 5500, 5000, 4500, 4000, 3500, 3000, 2500, 2000, 1500, 1000, 500, 400, 300, 250, 150, 100
               ]

# 4) Merge building features
train_building_feats = join_building_features(train_gdf_2263, buildings_2263, buffer_sizes, reproject_back=True)
test_building_feats  = join_building_features(test_gdf_2263, buildings_2263, buffer_sizes, reproject_back=True)

Processing KML building buffers:   0%|          | 0/35 [00:00<?, ?it/s]

current buffer being processed :15000


Processing KML building buffers:   3%|▎         | 1/35 [04:45<2:41:56, 285.78s/it]

current buffer being processed :14500


Processing KML building buffers:   6%|▌         | 2/35 [08:50<2:24:01, 261.86s/it]

...

Processing KML building buffers:  94%|█████████▍| 33/35 [05:44<00:01,  1.47it/s]

current buffer being processed :150


Processing KML building buffers:  97%|█████████▋| 34/35 [05:44<00:00,  1.75it/s]

current buffer being processed :100


Processing KML building buffers: 100%|██████████| 35/35 [05:44<00:00,  9.85s/it]
/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1528: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1528: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1528: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result o

In [13]:
test_features_df.columns

Index(['building_count_15000m_kml', 'building_area_15000m_kml',
       'building_area_min_15000m_kml', 'building_area_median_15000m_kml',
       'building_area_std_15000m_kml', 'building_count_15000m_kml_ring',
       'building_area_15000m_kml_ring', 'building_area_min_15000m_kml_ring',
       'building_area_median_15000m_kml_ring',
       'building_area_std_15000m_kml_ring',
       ...
       'building_count_100m_kml', 'building_area_100m_kml',
       'building_area_min_100m_kml', 'building_area_median_100m_kml',
       'building_area_std_100m_kml', 'building_count_100m_kml_ring',
       'building_area_100m_kml_ring', 'building_area_min_100m_kml_ring',
       'building_area_median_100m_kml_ring',
       'building_area_std_100m_kml_ring'],
      dtype='object', length=350)

In [9]:
cols_to_drop = ['Longitude', 'Latitude', 'datetime', 'UHI Index', 'geometry']
train_features_df = train_building_feats.drop(columns = cols_to_drop )

In [12]:
test_features_df = test_building_feats.drop(columns=['Longitude', 'geometry', 'Latitude', 'UHI Index'])

In [14]:
test_features_df.columns

Index(['building_count_15000m_kml', 'building_area_15000m_kml',
       'building_area_min_15000m_kml', 'building_area_median_15000m_kml',
       'building_area_std_15000m_kml', 'building_count_15000m_kml_ring',
       'building_area_15000m_kml_ring', 'building_area_min_15000m_kml_ring',
       'building_area_median_15000m_kml_ring',
       'building_area_std_15000m_kml_ring',
       ...
       'building_count_100m_kml', 'building_area_100m_kml',
       'building_area_min_100m_kml', 'building_area_median_100m_kml',
       'building_area_std_100m_kml', 'building_count_100m_kml_ring',
       'building_area_100m_kml_ring', 'building_area_min_100m_kml_ring',
       'building_area_median_100m_kml_ring',
       'building_area_std_100m_kml_ring'],
      dtype='object', length=350)

In [ ]:
# input_cols = ['building_area_12000m_kml', 'building_count_12000m_kml']
# train_final = pd.concat([train_gdf_features, train_features_df], axis=1)
# test_final  = pd.concat([test_gdf_features,  test_features_df], axis=1)

In [15]:
train_features_df.to_csv('train_kml.csv', index=False)
test_features_df.to_csv('test_kml.csv', index=False)

In [ ]:
# import xgboost as xgb
# target = train_df['UHI Index']
# xgb_kml = xgb.XGBRegressor()
# xgb_kml.fit(train_final, target)
# preds = xgb_kml.predict(test_final)

In [ ]:
# subm_test = pd.read_csv('/kaggle/input/test-df-ey-open-science/Submission_template_UHI2025-v2.csv')
# subm_test['UHI Index'] = preds
# subm_test.to_csv('ey_sub_kml.csv', index=False)

In [ ]:
# subm_test